---
## Practice Roadmap (Objects)
In this section you'll practice:
- Inspecting objects (`dir`, `getattr`, `callable`)
- Designing classes with `__repr__`, validation, and properties
- Class vs instance attributes
- Equality & ordering via `dataclass`
- Inheritance & polymorphism (abstract base classes)
- Composition (a `Ledger` managing accounts)
- Context managers (`__enter__` / `__exit__`)
- Iterables & `__iter__`
- `__slots__` for lightweight objects
- `copy` vs `deepcopy`
- Protocols (duck typing)


In [1]:
class Account:
    def __init__(self, account_number, account_type, initial_balance):
        self.account_number = account_number
        self.account_type = account_type
        self.balance = initial_balance
        
    def deposit(self, amount):
        # should also check that amount is a numerical value!
        if amount > 0:
            self.balance = self.balance + amount
            print(f'Deposited {amount}')
            print(f'New balance is: {self.balance}')
        else:
            print(f'{amount} is an invalid amount.')
            
    def withdraw(self, amount):
        # should also check that amount is a numerical value!
        if amount > 0 and amount <= self.balance:
            self.balance = self.balance - amount
            print(f'Withdrawal: {amount}')
            print(f'New Balance: {self.balance}')
        else:
            if amount < 0:
                print(f'{amount} is an invalid amount')
            else:
                print('Insufficient funds.')
                print(f'Current balance is {self.balance}')

In [2]:
my_account = Account('123-456', 'savings', 1_000.00)

### 1) Inspect object state & behavior
Use built-ins to discover what an object can do.


In [3]:
# Using your existing my_account from above
names = [n for n in dir(my_account) if not n.startswith('_')]
print("Public-ish attributes/methods:", names)
print("Has attribute 'balance'?", hasattr(my_account, 'balance'))
print("deposit is callable?", callable(getattr(my_account, 'deposit', None)))


Public-ish attributes/methods: ['account_number', 'account_type', 'balance', 'deposit', 'withdraw']
Has attribute 'balance'? True
deposit is callable? True


### 2) A safer money class: `BankAccount`
- Uses `Decimal` (avoid float rounding for money)
- Validates amounts
- Exposes `balance` via a read-only property
- Friendly `__repr__`
- `classmethod` constructor
- `staticmethod` validator


In [4]:
from decimal import Decimal, ROUND_HALF_UP

class BankAccount:
    bank_name = "PyBank"  # class attribute (shared default)

    def __init__(self, number: str, kind: str, initial_balance: "Decimal|int|str|float" = 0):
        self.number = number
        self.kind = kind
        self._balance = self._to_decimal(initial_balance)

    def __repr__(self):
        return f"BankAccount(number={self.number!r}, kind={self.kind!r}, balance={self._balance})"

    @staticmethod
    def _to_decimal(x):
        # Convert safely via str for floats/ints
        return Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)

    @staticmethod
    def _validate_amount(amount):
        d = BankAccount._to_decimal(amount)
        if d <= 0:
            raise ValueError("amount must be > 0")
        return d

    @property
    def balance(self) -> Decimal:
        return self._balance

    def deposit(self, amount):
        d = self._validate_amount(amount)
        self._balance += d
        return self  # enable chaining

    def withdraw(self, amount):
        d = self._validate_amount(amount)
        if d > self._balance:
            raise ValueError("insufficient funds")
        self._balance -= d
        return self

    @classmethod
    def from_eur(cls, number: str, kind: str, eur):
        return cls(number, kind, eur)

# Quick checks
acct = BankAccount.from_eur("BE-001", "savings", 1000)
acct.deposit(50).withdraw("12.34")
assert str(acct.balance) == "1037.66"
repr(acct)
print("OK ✅", acct)


OK ✅ BankAccount(number='BE-001', kind='savings', balance=1037.66)


### 3) Class vs instance attributes
Show how instances can shadow class attributes.


In [5]:
a = BankAccount("X-1", "checking", 10)
b = BankAccount("X-2", "checking", 10)
print(BankAccount.bank_name, a.bank_name, b.bank_name)
a.bank_name = "MyOwnBank"  # shadows class attr on a only
print("After shadow:", BankAccount.bank_name, a.bank_name, b.bank_name)
del a.bank_name
print("After delete shadow:", a.bank_name)


PyBank PyBank PyBank
After shadow: PyBank MyOwnBank PyBank
After delete shadow: PyBank


### 4) Equality & ordering with `@dataclass`
Use dataclasses to get `__init__`, `__repr__`, `__eq__`, and ordering for free.


In [6]:
from dataclasses import dataclass, field
from datetime import datetime

@dataclass(order=True)
class AccountRecord:
    sort_index: datetime = field(init=False, repr=False, compare=True)
    number: str
    owner: str
    opened: datetime
    balance: Decimal

    def __post_init__(self):
        self.sort_index = self.opened

recs = [
    AccountRecord("A-2", "Bea", datetime(2024, 1, 1), Decimal("200.00")),
    AccountRecord("A-1", "Alex", datetime(2023, 6, 1), Decimal("300.00")),
    AccountRecord("A-3", "Cara", datetime(2025, 3, 1), Decimal("100.00")),
]
print("Unsorted:")
for r in recs: print(r)
print("\nSorted by opened (order=True):")
for r in sorted(recs): print(r)
assert sorted(recs)[0].number == "A-1"
print("OK ✅")


Unsorted:
AccountRecord(number='A-2', owner='Bea', opened=datetime.datetime(2024, 1, 1, 0, 0), balance=Decimal('200.00'))
AccountRecord(number='A-1', owner='Alex', opened=datetime.datetime(2023, 6, 1, 0, 0), balance=Decimal('300.00'))
AccountRecord(number='A-3', owner='Cara', opened=datetime.datetime(2025, 3, 1, 0, 0), balance=Decimal('100.00'))

Sorted by opened (order=True):
AccountRecord(number='A-1', owner='Alex', opened=datetime.datetime(2023, 6, 1, 0, 0), balance=Decimal('300.00'))
AccountRecord(number='A-2', owner='Bea', opened=datetime.datetime(2024, 1, 1, 0, 0), balance=Decimal('200.00'))
AccountRecord(number='A-3', owner='Cara', opened=datetime.datetime(2025, 3, 1, 0, 0), balance=Decimal('100.00'))
OK ✅


### 5) Inheritance & polymorphism
Define a small hierarchy and operate on a mixed list polymorphically.


In [7]:
from abc import ABC, abstractmethod

class AccountBase(ABC):
    @abstractmethod
    def deposit(self, amount): ...
    @abstractmethod
    def withdraw(self, amount): ...

class SavingsAccount(BankAccount, AccountBase):
    pass  # same as BankAccount for now

class CheckingAccount(BankAccount, AccountBase):
    FEE = Decimal("0.50")
    def withdraw(self, amount):
        super().withdraw(amount)
        # apply flat fee
        if self.balance < self.FEE:
            raise ValueError("insufficient for fee")
        self._balance -= self.FEE
        return self

accts: list[AccountBase] = [
    SavingsAccount("S-1", "savings", 100),
    CheckingAccount("C-1", "checking", 100),
]
for a in accts:
    a.deposit(10).withdraw(5)
print([(type(a).__name__, str(a.balance)) for a in accts])
assert str(accts[0].balance) == "105.00"
assert str(accts[1].balance) == "104.50"  # 100+10-5-0.50
print("OK ✅")


[('SavingsAccount', '105.00'), ('CheckingAccount', '104.50')]
OK ✅


### 6) Composition: a `Ledger` that manages accounts & history
Prefer composition to inheritance when modeling "has-a" relationships.


In [8]:
from datetime import datetime

class Ledger:
    def __init__(self):
        self._accounts: dict[str, BankAccount] = {}
        self._history: list[tuple[datetime, str]] = []  # (timestamp, message)

    def add(self, acct: BankAccount):
        if acct.number in self._accounts:
            raise KeyError("duplicate account number")
        self._accounts[acct.number] = acct
        self._log(f"added {acct.number}")

    def post(self, number: str, action: str, amount):
        acct = self._accounts[number]
        getattr(acct, action)(amount)
        self._log(f"{action} {amount} on {number}")

    def _log(self, msg: str):
        self._history.append((datetime.utcnow(), msg))

    def history(self):
        return list(self._history)

ledger = Ledger()
ledger.add(SavingsAccount("S-42", "savings", 50))
ledger.post("S-42", "deposit", 25)
ledger.post("S-42", "withdraw", 10)
print(ledger.history()[-2:])
assert "deposit" in ledger.history()[1][1]
print("OK ✅")


[(datetime.datetime(2025, 10, 11, 16, 14, 20, 650480), 'deposit 25 on S-42'), (datetime.datetime(2025, 10, 11, 16, 14, 20, 650565), 'withdraw 10 on S-42')]
OK ✅


C:\Users\user1\AppData\Local\Temp\ipykernel_9084\237361166.py:20: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self._history.append((datetime.utcnow(), msg))


### 7) Context manager: atomic transaction with rollback on error
Use `__enter__`/`__exit__` to ensure state safety.


In [9]:
class Transaction:
    def __init__(self, acct: BankAccount):
        self.acct = acct
        self._snapshot = acct.balance
    def __enter__(self):
        return self.acct
    def __exit__(self, exc_type, exc, tb):
        if exc:
            # Roll back
            self.acct._balance = self._snapshot
            print("Rolled back due to:", exc)
            return True  # suppress exception
        return False

acct2 = BankAccount("TX-1", "checking", 100)
with Transaction(acct2) as a:
    a.deposit(50)
    a.withdraw(200)  # will raise, then rollback
print("After tx:", acct2.balance)
assert str(acct2.balance) == "100.00"
print("OK ✅")


Rolled back due to: insufficient funds
After tx: 100.00
OK ✅


### 8) Iterables: make a simple history object you can loop over
Implement `__iter__` to support `for ... in ...`.


In [10]:
class History:
    def __init__(self):
        self._events: list[str] = []
    def add(self, msg: str):
        self._events.append(msg)
    def __iter__(self):
        yield from self._events

h = History()
h.add("opened")
h.add("deposit 20")
for ev in h:
    print("event:", ev)
assert list(h) == ["opened", "deposit 20"]
print("OK ✅")


event: opened
event: deposit 20
OK ✅


### 9) `__slots__` for lightweight objects
Restrict attributes to save memory & catch typos at runtime.


In [11]:
class LightweightPoint:
    __slots__ = ("x", "y")
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = LightweightPoint(1, 2)
try:
    p.z = 3  # AttributeError (not in __slots__)
except AttributeError as e:
    print("Caught:", e)
print("OK ✅")


Caught: 'LightweightPoint' object has no attribute 'z' and no __dict__ for setting new attributes
OK ✅


### 10) `copy` vs `deepcopy`
Understand how nested state behaves when copying objects.


In [12]:
import copy

class Portfolio:
    def __init__(self, accounts: list[BankAccount]):
        self.accounts = accounts  # shared list reference!

ba = BankAccount("P-1", "savings", 10)
pb1 = Portfolio([ba])
pb2 = copy.copy(pb1)       # shallow copy
pb3 = copy.deepcopy(pb1)   # deep copy

pb1.accounts[0].deposit(5)
print("shallow sees change?", pb2.accounts[0].balance)
print("deep sees change?  ", pb3.accounts[0].balance)
assert str(pb2.accounts[0].balance) == "15.00"
assert str(pb3.accounts[0].balance) == "10.00"
print("OK ✅")


shallow sees change? 15.00
deep sees change?   10.00
OK ✅


### 11) Protocols (duck typing)
Use `Protocol` to type-check behavior without inheritance coupling.


In [13]:
from typing import Protocol, Iterable

class SupportsDeposit(Protocol):
    def deposit(self, amount): ...

def credit_all(items: Iterable[SupportsDeposit], amount):
    for it in items:
        it.deposit(amount)

class Piggy:
    def __init__(self): self.total = Decimal("0.00")
    def deposit(self, amount): self.total += BankAccount._to_decimal(amount)

pa = Piggy()
ba = BankAccount("PR-1", "savings", 0)
credit_all([pa, ba], 2.5)
assert str(pa.total) == "2.50" and str(ba.balance) == "2.50"
print("OK ✅")


OK ✅


### Bonus exercises (optional)
1. Add a `transfer(to_account, amount)` method to `BankAccount` that is atomic (use the `Transaction` context manager).
2. Make `BankAccount` hashable by account `number` only (`__hash__`, `__eq__`). Discuss pros/cons.
3. Add an `InterestBearing` mixin with `apply_interest(rate)` and mix it into `SavingsAccount` only; test polymorphically.
4. Turn `History` into a reversible iterable: implement `__reversed__`.
5. Use `@dataclass(frozen=True)` to create an immutable `Customer` and show that attribute reassignment raises.
